<a href="https://colab.research.google.com/github/aarmintia/imdb/blob/main/Data_Strategist_GCash_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reference:
https://medium.com/analytics-vidhya/exploratory-data-analysis-imdb-dataset-cff0c3991ad5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_palette("viridis")
import pickle
import glob
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Title Basics Table

In [ ]:
df_title_basics0 = pickle.load(open("/content/drive/MyDrive/extracted/title.basics.sav","rb"))

In [ ]:
df_title_basics_copy = df_title_basics0.copy()

In [ ]:
df_title_basics = df_title_basics_copy[(df_title_basics_copy.titleType == "movie") | (df_title_basics_copy.titleType == "tvMovie")]

In [ ]:
df_title_basics_2020 = df_title_basics[df_title_basics['startYear'] == 2020]
df_title_basics_2020.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
61107,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0.0,2020.0,NaN,70,Drama
220830,tt0230622,movie,Parinati,Parinati,0.0,2020.0,NaN,125,Drama
280921,tt0293513,movie,"Pit Bull: A Tale of Lust, Murder and Revenge",Pit Bull,0.0,2020.0,NaN,NaN,NaN
352668,tt0368133,movie,The Promise of Perfume,The Promise of Perfume,0.0,2020.0,NaN,57,Documentary
420996,tt0438755,movie,About Face: The Story of the Jewish Refugee So...,About Face: The Story of the Jewish Refugee So...,0.0,2020.0,NaN,94,Documentary


# Title Principals Table

In [ ]:
import pandas as pd
import pickle
import glob

def load_and_concatenate_chunks(file_pattern):
    files = glob.glob(file_pattern)
    dfs = []
    for file in files:
        df_chunk = pickle.load(open(file, "rb"))
        dfs.append(df_chunk)
    return pd.concat(dfs, ignore_index=True)

df_title_principals0 = load_and_concatenate_chunks("/content/drive/MyDrive/extracted/title.principals_chunk_*.sav")


In [ ]:
df_title_principals = df_title_principals0.copy()

df_title_principals = df_title_principals[(df_title_principals.category == "actor") | (df_title_principals.category == "actress")]
#df_title_principals.to_csv('filtered_title_principals.tsv', sep='\t', index=False)
df_title_principals.head()

,tconst,nconst,category
11,tt0000005,nm0443482,actor
12,tt0000005,nm0653042,actor
16,tt0000007,nm0179163,actor
17,tt0000007,nm0183947,actor
21,tt0000008,nm0653028,actor


# Name Basics Table

In [ ]:
df_name_basics = pickle.load(open("/content/drive/MyDrive/extracted/name.basics.sav","rb"))

In [ ]:
df_name_basics_copy = df_name_basics.copy()
df_name_basics_copy.drop(["birthYear","deathYear","primaryProfession","knownForTitles"],axis=1,inplace=True)
df_name_basics_copy.head(1)

# Ratings Table

In [ ]:
df_ratings = pickle.load(open("/content/drive/MyDrive/extracted/title.ratings.sav","rb"))
df_ratings.head(1)

,tconst,averageRating,numVotes
0,tt0000001,5.7,2033


Merge name to principal table

In [ ]:
merged_principal_name = pd.merge(df_title_principals, df_name_basics_copy, on='nconst', how='inner')
merged_principal_name.head()

In [ ]:
#merged_principal_name.to_csv('merged_principal_name.tsv', sep='\t', header=True)

Merge rating

In [ ]:
merged_principal_name = pd.read_csv("/content/merged_principal_name.tsv", sep='\t')

In [ ]:
merged_principal_name_rating = pd.merge(merged_principal_name, df_ratings, on='tconst', how='inner')


In [ ]:
merged_principal_name_rating.head(1)

,Unnamed: 0,tconst,nconst,category,primaryName,averageRating,numVotes
0,0,tt0000005,nm0443482,actor,Charles Kayser,6.2,2733


In [ ]:
# merged_principal_name_rating.to_csv('merged_principal_name_rating.tsv', sep='\t', header=True)

In [ ]:
merged_principal_name_rating_basic = pd.merge(merged_principal_name_rating, df_title_basics, on='tconst', how='inner')

In [ ]:
merged_principal_name_rating_basic.head()

,Unnamed: 0,tconst,nconst,category,primaryName,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,6,tt0011603,nm0179163,actor,James J. Corbett,3.5,60,movie,The Prince of Avenue A,The Prince of Avenue A,0.0,1920.0,NaN,50,Drama
1,65386,tt0011603,nm0089162,actor,Frederick Vroom,3.5,60,movie,The Prince of Avenue A,The Prince of Avenue A,0.0,1920.0,NaN,50,Drama
2,131186,tt0011603,nm0237642,actress,Cora Drew,3.5,60,movie,The Prince of Avenue A,The Prince of Avenue A,0.0,1920.0,NaN,50,Drama
3,133427,tt0011603,nm0191947,actor,Richard Cummings,3.5,60,movie,The Prince of Avenue A,The Prince of Avenue A,0.0,1920.0,NaN,50,Drama
4,201392,tt0011603,nm0912951,actress,Mary Warren,3.5,60,movie,The Prince of Avenue A,The Prince of Avenue A,0.0,1920.0,NaN,50,Drama


In [ ]:
merged_principal_name_rating_basic.to_csv('combined_table.tsv', sep='\t', header=True)

In [ ]:
combined_table = pd.read_csv("/content/combined_table.tsv", sep='\t')

In [ ]:
combined_table['tconst'].nunique()

309084

In [ ]:
import pandas as pd

actor_actress_data = combined_table[(combined_table['category'].isin(['actor', 'actress'])) & (combined_table['startYear'] == 2020)]

actor_actress_counts = actor_actress_data.groupby(['category', 'primaryName'])['tconst'].nunique().reset_index()
actor_actress_counts.columns = ['Category', 'Actor/Actress', 'Unique_tconst_Count']

actor_actress_counts_sorted = actor_actress_counts.sort_values(by='Unique_tconst_Count', ascending=False)

actor_actress_total_votes = actor_actress_data.groupby(['category', 'primaryName'])['numVotes'].sum().reset_index()
actor_actress_total_votes.columns = ['Category', 'Actor/Actress', 'Total Votes']

actor_actress_summary = pd.merge(actor_actress_counts_sorted, actor_actress_total_votes, on=['Category', 'Actor/Actress'], how='inner')

print(actor_actress_summary)


      Category         Actor/Actress  Unique_tconst_Count  Total Votes
0        actor          Eric Roberts                   16         2611
1        actor            Simon Hill                   13          305
2      actress        Kelsey Painter                   10          231
3      actress  Mehazabien Chowdhury                   10          295
4        actor           Afran Nisho                    9         1532
...        ...                   ...                  ...          ...
26863    actor       Marc Villanueva                    1           15
26864    actor           Marc Victor                    1          342
26865    actor          Marc Titieni                    1          209
26866    actor           Marc Terrés                    1           41
26867  actress         Ülle Kaljuste                    1          907

[26868 rows x 4 columns]


In [ ]:
import pandas as pd

actor_actress_data = combined_table[(combined_table['category'].isin(['actor', 'actress'])) & (combined_table['startYear'] == 2020)]

actor_actress_counts = actor_actress_data.groupby(['category', 'primaryName'])['tconst'].nunique().reset_index()
actor_actress_counts.columns = ['Category', 'Actor/Actress', 'Unique_tconst_Count']

actor_actress_total_votes = actor_actress_data.groupby(['category', 'primaryName'])['numVotes'].sum().reset_index()
actor_actress_total_votes.columns = ['Category', 'Actor/Actress', 'Total Votes']

actor_actress_total_votes_sorted = actor_actress_total_votes.sort_values(by='Total Votes', ascending=False)

actor_actress_summary = pd.merge(actor_actress_total_votes_sorted, actor_actress_counts, on=['Category', 'Actor/Actress'], how='inner')

print(actor_actress_summary)


      Category          Actor/Actress  Total Votes  Unique_tconst_Count
0        actor           Juhan Ulfsak       585121                    1
1        actor       Robert Pattinson       585121                    1
2        actor  John David Washington       585121                    1
3      actress      Elizabeth Debicki       585121                    1
4        actor             Jamie Foxx       472743                    2
...        ...                    ...          ...                  ...
26863    actor       Maximiliano Zago            5                    1
26864  actress         Kayla Willmann            5                    1
26865    actor         Robert Madison            5                    1
26866    actor            Igor Horvat            5                    1
26867    actor             Nick Peres            5                    1

[26868 rows x 4 columns]


In [ ]:
combined_table[(combined_table['primaryName'] == 'Simon Hill') & (combined_table['startYear'] == 2020)]

,Unnamed: 0.1,Unnamed: 0,tconst,nconst,category,primaryName,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
895519,895519,14484799,tt12969746,nm2993483,actor,Simon Hill,1.4,6,movie,Welcome to the Day of the Dead,Welcome to the Day of the Dead,0.0,2020.0,NaN,68.0,Family
1006135,1006135,14484795,tt12404028,nm2993483,actor,Simon Hill,4.0,9,movie,The Lion Kid 2,The Lion Kid 2,0.0,2020.0,NaN,73.0,"Animation,Family,Western"
1179007,1179007,14484798,tt12969558,nm2993483,actor,Simon Hill,2.0,7,movie,Lil' Monsters 2,Lil' Monsters 2,0.0,2020.0,NaN,70.0,Drama
1179067,1179067,14484794,tt12375526,nm2993483,actor,Simon Hill,6.6,61,movie,Pets,Pets,0.0,2020.0,NaN,60.0,"Animation,Comedy,Family"
1179073,1179073,19296946,tt12981770,nm12931861,actor,Simon Hill,2.7,18,movie,Witchmas,Witchmas,0.0,2020.0,NaN,70.0,Animation
1179187,1179187,14484789,tt11749986,nm2993483,actor,Simon Hill,4.1,33,movie,Spaced Cats,Spaced Cats,0.0,2020.0,NaN,69.0,"Animation,Comedy,Sci-Fi"
1179191,1179191,14484792,tt12358316,nm2993483,actor,Simon Hill,4.5,61,movie,Little Foot,Little Foot,0.0,2020.0,NaN,72.0,"Animation,Family"
1179196,1179196,14484793,tt12369756,nm2993483,actor,Simon Hill,5.2,14,movie,Avenger Dogs 2: Wonder Dogs,Avenger Dogs 2: Wonder Dogs,0.0,2020.0,NaN,70.0,"Animation,Comedy,Family"
1179202,1179202,14484796,tt12416788,nm2993483,actor,Simon Hill,3.9,19,movie,00K9,00K9,0.0,2020.0,NaN,70.0,"Action,Animation,Comedy"
1179208,1179208,14484800,tt12969810,nm2993483,actor,Simon Hill,3.2,9,movie,Dave of the Dead,Dave of the Dead,0.0,2020.0,NaN,70.0,Animation
